In [1]:
import pandas as pd
import numpy as np

In [2]:
lv_df = pd.read_csv('/Users/ethanweber/Downloads/LV_df.csv')

/Users/ethanweber/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,4,6,17,18,32,34,36,38,40) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
def fix_categories(x):
    x = str(x)
    x = x.replace('[', '').replace(']', '').replace(',', '').split("u'")
    x = map(lambda y: y.replace("'", ""), x)
    x = map(lambda y: y.replace('u"', "").replace('"', "").strip(), x)
    if filter(None, x) != None: 
        return filter(None, x)
    else:
        return x

lv_df['categories'] = map(fix_categories, lv_df['categories'])

In [4]:
def get_big_cat(z):
    catlist = ['Active Life', 'Arts & Entertainment', 'Automotive', 'Beauty & Spas', 'Education', 'Event Planning & Services', 'Financial Services', 'Food', 'Health & Medical', 'Home Services', 'Hotels & Travel', 'Local Flavor', 'Local Services',  'Nightlife', 'Pets', 'Professional Services', 'Public Services & Government', 'Real Estate', 'Religious Organizations', 'Restaurants', 'Shopping']
    return filter(lambda x: x in catlist, z)
lv_df['big_cat'] = map(get_big_cat, lv_df['categories'])

In [5]:
def get_little_cat(z):
    catlist = ['Active Life', 'Arts & Entertainment', 'Automotive', 'Beauty & Spas', 'Education', 'Event Planning & Services', 'Financial Services', 'Food', 'Health & Medical', 'Home Services', 'Hotels & Travel', 'Local Flavor', 'Local Services',  'Nightlife', 'Pets', 'Professional Services', 'Public Services & Government', 'Real Estate', 'Religious Organizations', 'Restaurants', 'Shopping']
    return filter(lambda x: (x in catlist) == False, z)

lv_df['categories'] = map(get_little_cat, lv_df['categories'])

In [ ]:
def get_excluder(z):
    rmlist = ['Automotive', 'Education', 'Financial Services', 'Mass Media', 'Home Services', 'Local Services', 'Pets', 'Professional Services', 'Public Services & Government', 'Real Estate', 'Religious Organizations']
    for i in rmlist:
        if i in z:
            return 1
    return 0

#lv_df['rm'] = map(get_excluder, lv_df['big_cat'])

In [ ]:
lv_df = lv_df[lv_df['rm'] == 0]


In [ ]:
lv_df.categories[0]

In [ ]:
def nested(x):
    for i in x:
        if i == None or i =='':
            pass
        elif i[-len(' True,'):] == ' True,':
            print i[:-len(': True,')]
        elif i[-len(' False,'):] == ' False,':
            pass

In [ ]:
lv_df2[]

In [ ]:
lv_df2 = lv_df
#del lv_df2['index']
del lv_df2['Unnamed: 0']

In [ ]:
lv_df2.text2

In [ ]:
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import euclidean_distances

import nltk
from nltk.tag import pos_tag
from nltk import word_tokenize
from nltk.corpus import stopwords
stop = stopwords.words('english')
def cleaning_text(sentence):
    sentence = str(sentence)
    sentence=sentence.lower()
    sentence=re.sub('[^\w\s]',' ', sentence) #removes punctuations
    sentence=re.sub('_',' ', sentence) #removes punctuations
    sentence=re.sub('\d+',' ', sentence) #removes digits
    cleaned=' '.join([w for w in sentence.split() if not w in stop]) # removes english stopwords
    cleaned=' '.join([w for w , pos in pos_tag(cleaned.split()) if (pos == 'NN' or pos=='JJ' or pos=='JJR' or pos=='JJS' )])
    
#selecting only nouns and adjectives
    cleaned=' '.join([w for w in cleaned.split() if not len(w)<=2 ]) #removes single lettered words and digits
    cleaned=cleaned.strip()
    return cleaned

#lv_df['text'] = lv_df['text'].apply(lambda x: cleaning_text(x))

In [ ]:
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import euclidean_distances
import gensim
import nltk
from nltk.tag import pos_tag
from nltk import word_tokenize
from nltk.corpus import stopwords
stop = stopwords.words('english')

lmtizer=nltk.stem.WordNetLemmatizer()
stemmer = nltk.stem.PorterStemmer() # Create our stemmer 
stem_lem = lambda x: lmtizer.lemmatize(stemmer.stem(x))

In [ ]:
def split_stem_lem(x):
    try:
        x = map(lambda y: stem_lem(y), cleaning_text(x).split())
        return x
    except:
        print 'Fuck'
        return None
    


In [ ]:
def fix_big_cat(x):
    return x.replace('[', '').replace(']', '').split(',')


In [ ]:
import pandas as pd
lv_df = pd.read_csv('lv_df_final.csv', sep='|', encoding='utf-8')

In [ ]:
def fix_attributes(x):
    x = str(x)
    x = sum(map(lambda x: x.split('u"'), x.replace('[', '').replace(']', '').split("u'")), [])
    x = map(lambda y: y.replace("'", "").replace('"', ""), x)
    x = map(lambda y: y.replace('u"', "").replace('"', "").strip(), x)
    if filter(None, x) != None: 
        x = nested(x)
        return filter(None, x)
    else:
        return x
    
    
def ambiance_other(x):
    x = x.remove('{').remove('}')
    left, right = x.items()[0]
    right = right[:-2]
    right = right.split(',')
    right = map(lambda x: x.split(':'), right)
    right = map(lambda x: x[0].strip() if (x[1].strip() == 'True') else None, right)
    #left = x[:-2]#.split(',')
    #left = map(lambda x: x.split(':'), left)
    return {left : filter(lambda x: x!= None, right)}



    
def truevals(x):
    try:
        z = x.split(':', 1)
        #print z
        if z[1] == ' True,':
            return z[0]
        elif z[1] == ' False,':
            pass
        elif z[1] == ' no,':
            pass
        elif z[1].replace('{', "").strip() == 'no':
            pass
        elif (z[1][-2:] == '},'):
            z2 = map(lambda x: x.strip().split(':'), z[1].replace('{', '').replace('}', '').split(','))
            return {z[0] : map(lambda x: x[0].strip().replace(',', ''), filter(lambda x: ' True' in x, z2))}
            #z = {z[0]: filter(lambda x: x!= None, map(lambda x: x[0] if (x[1] == ' True') else None, z2))}
        elif (z[1][-1:] == '}'):
            z2 = map(lambda x: x.strip().split(':'), z[1].replace('{', '').replace('}', '').split(','))
            return {z[0] : map(lambda x: x[0].strip().replace(',', ''), filter(lambda x: ' True' in x, z2))}
            #z = {z[0]: filter(lambda x: x!= None, map(lambda x: x[0] if (x[1] == ' True') else None, z2))}
        else:
            return {z[0] : z[1].replace('{', "").strip().replace(',', '')}
    except:
        return None
    
    




def nested(x):
    out = []
    for i in x:
        j = truevals(i)
        if j != None:
            out.append(truevals(i))
            
        #elif i[-len(' True,'):] == ' True,':
        #    i = i[:-len(': True,')]
        #elif i[-len(' False,'):] == ' False,':
        #    pass
        #else:
        #    i = i.split(':', 1)
        #    print i
        #if j != ['']:
        #    try:
        #        if j[1][0] == '{':
        #            j = {j[0].strip() : j[1].strip()}
        #            print j
        #    except:
        #        pass
    return out


        
main_att = lambda x: list(filter(lambda x: x != None, fix_attributes(x)))


In [ ]:
lv_df['attributes'] = map(main_att, lv_df['attributes'])

In [ ]:
lv_df['attributes'][1]

In [ ]:
def fix_text2(x):
    x = str(x)
    x = x.replace('[', '').replace(']', '').replace(',', '').split(" ")
    x = map(lambda y: y.replace("'", ""), x)
    x = map(lambda y: y.replace('u"', "").replace('"', "").strip(), x)
    if filter(None, x) != None: 
        return filter(None, x)
    else:
        return x

In [ ]:
import pandas as pd
import numpy as np

print 'going...'
lv_df = pd.read_csv('lv_df_final.csv', sep='|', encoding='utf-8')
print 'fixing text2...'
lv_df['text2'] = map(fix_text2, lv_df['text2'])
print 'fixed text2..'
def fix_cat(x):
    return str(x).replace('[', '').replace(']', '').split(',')
print 'fixing big_cat'
lv_df['big_cat'] = map(fix_cat, lv_df['big_cat'])
print 'fixing categories'
lv_df['categories'] = map(fix_cat, lv_df['categories'])

In [ ]:
del lv_df['text']

In [ ]:
lv_df.attributes

## flattened = [val for sublist in list(lv_df2['text2']) for val in sublist]

In [ ]:
tokens = flattened

In [ ]:
corpus = list(lv_df['text2'])

In [ ]:
clipped_corpus = gensim.utils.ClippedCorpus(corpus, max_docs=None) 
id2word = gensim.corpora.Dictionary.load_from_text('wiki_en_wordids.txt')
lsi = gensim.models.lsimodel.LsiModel(corpus=clipped_corpus, id2word=id2word, num_topics=20)

In [ ]:
print clipped_corpus

In [ ]:
def recover_cat_list(x):
    x = str(x).replace('[', '').replace(']', '').split(',')
    x = map(lambda y: y.replace("'", ""), x)
    x = map(lambda y: y.replace('u"', "").replace('"', "").strip(), x)
    if filter(None, x) != None: 
        return filter(None, x)
    else:
        return x

In [ ]:
lv_df['categories'] = map(recover_cat_list, lv_df['categories'])

In [ ]:
lv_df['big_cat'] = map(recover_cat_list, lv_df['big_cat'])

In [ ]:
import logging
import os
import sys
import re
import tarfile
import itertools


import nltk
from nltk.collocations import TrigramCollocationFinder
from nltk.metrics import BigramAssocMeasures, TrigramAssocMeasures


import gensim
from gensim.parsing.preprocessing import STOPWORDS

def flatten(x):
    return sum(list(x), [])

def tokenize(text):
    return [token for token in gensim.utils.simple_preprocess(text) if token not in STOPWORDS]

In [ ]:
col_list = list(lv_df2['text2'])
#word_list = sum(, [])

In [ ]:
vector = next(iter(tokens))

In [6]:
restaurant_df = lv_df[map(lambda x: 'Restaurants' in x, lv_df['big_cat'])]

In [7]:
restaurant_df = restaurant_df[restaurant_df.is_open == 1.0]

In [8]:
rest_df = restaurant_df.drop_duplicates('business_id')

In [12]:
del rest_df['Unnamed: 0']

In [13]:
master_list = sum(list(rest_df.categories), [])

In [14]:
d = {}

In [15]:
for i in list(set(master_list)):
    d[i] = master_list.count(i)

In [16]:
bc_list = sum(list(rest_df.big_cat), [])

In [17]:
for i in list(set(bc_list)):
    if i == 'Restaurants':
        pass
    else:
        d[i] = bc_list.count(i)

In [19]:
z = 0
cat_list = []
for i in d:
    if d[i] >= 12:
        cat_list.append(i)


In [20]:
cat_list

['Buffets',
 'Creperies',
 'Juice Bars & Smoothies',
 'Grocery',
 'Delis',
 'Halal',
 'Cantonese',
 'Noodles',
 'Chicken Wings',
 'Food Stands',
 'Lounges',
 'Cajun/Creole',
 'Greek',
 'Chinese',
 'Food Trucks',
 'Middle Eastern',
 'Diners',
 'Sports Bars',
 'Vegetarian',
 'Chicken Shop',
 'Taiwanese',
 'Vietnamese',
 'Pizza',
 'Poke',
 'Food Court',
 'Music Venues',
 'Seafood',
 'Japanese',
 'Hot Dogs',
 'Salvadoran',
 'Sandwiches',
 'Dim Sum',
 'Nightlife',
 'Cocktail Bars',
 'Dive Bars',
 'Active Life',
 'Latin American',
 'Coffee & Tea',
 'Cheesesteaks',
 'Tapas Bars',
 'Szechuan',
 'Burgers',
 'Indian',
 'Bubble Tea',
 'Wine & Spirits',
 'Barbeque',
 'Party & Event Planning',
 'Food',
 'Venues & Event Spaces',
 'Ethnic Food',
 'Comfort Food',
 'Soul Food',
 'Gluten-Free',
 'Arts & Entertainment',
 'Sushi Bars',
 'French',
 'Food Delivery Services',
 'Italian',
 'Mediterranean',
 'Tex-Mex',
 'Dance Clubs',
 'American (New)',
 'Caterers',
 'Breakfast & Brunch',
 'Shopping',
 'Hawaii

In [24]:
cat_df = pd.DataFrame(rest_df.iloc[:,[0, 12, 47]])


In [28]:
cat_df = cat_df.reset_index()
cat_df

,index,business_id,categories,big_cat
0,0,CiYLq33nAyghFkUR15pP-Q,[Mexican],[Restaurants]
1,1,P7pxQFqr7yBKMMI2J51udw,"[American (New), Burgers, Bars, American (Trad...","[Restaurants, Nightlife]"
2,2,g8OnV26ywJlZpezdBnOWUQ,"[Breakfast & Brunch, American (New)]",[Restaurants]
3,4,gTlDDzDEHyDQ6iwjNhpI6A,[Indian],[Restaurants]
4,5,-WLrZPzjKfrftLWaCi1QZQ,"[Breakfast & Brunch, Diners]",[Restaurants]
5,6,3LWsVfsSmb_Nzbi2YQ-NIA,"[Halal, Vegetarian, Indian, Buffets, Food Deli...","[Restaurants, Food]"
6,7,AnXovUgUHJap3QaDRnjL1w,"[Diners, American (New), Breakfast & Brunch]",[Restaurants]
7,11,d_L-rfS1vT3JMzgCUGtiow,"[Tapas/Small Plates, Breakfast & Brunch, Mexican]",[Restaurants]
8,12,LYWbyu1EQM2o5hwHG5_Ldg,"[Sushi Bars, Seafood]",[Restaurants]
9,16,ujHiaprwCQ5ewziu0Vi9rw,"[American (New), Buffets]",[Restaurants]


In [30]:
del cat_df['index']
cat_df.columns

Index([u'business_id', u'categories', u'big_cat'], dtype='object')

In [32]:
for i in cat_list:
    print i
    cat_df[i] = pd.Series([0]*len(cat_df))

Buffets
Creperies
Juice Bars & Smoothies
Grocery
Delis
Halal
Cantonese
Noodles
Chicken Wings
Food Stands
Lounges
Cajun/Creole
Greek
Chinese
Food Trucks
Middle Eastern
Diners
Sports Bars
Vegetarian
Chicken Shop
Taiwanese
Vietnamese
Pizza
Poke
Food Court
Music Venues
Seafood
Japanese
Hot Dogs
Salvadoran
Sandwiches
Dim Sum
Nightlife
Cocktail Bars
Dive Bars
Active Life
Latin American
Coffee & Tea
Cheesesteaks
Tapas Bars
Szechuan
Burgers
Indian
Bubble Tea
Wine & Spirits
Barbeque
Party & Event Planning
Food
Venues & Event Spaces
Ethnic Food
Comfort Food
Soul Food
Gluten-Free
Arts & Entertainment
Sushi Bars
French
Food Delivery Services
Italian
Mediterranean
Tex-Mex
Dance Clubs
American (New)
Caterers
Breakfast & Brunch
Shopping
Hawaiian
Thai
Beer Bar
Asian Fusion
Southern
Ramen
Casinos
Wine Bars
Hotels
Bagels
Salad
Local Flavor
Hotels & Travel
Soup
American (Traditional)
Street Vendors
Tapas/Small Plates
Karaoke
Filipino
Steakhouses
Desserts
Event Planning & Services
Beer
Cafes
Ice Cream & F

In [33]:
for i in range(len(cat_df)):
    if (i % 100 == 0):
        print i
    cats = (cat_df.big_cat[i] + cat_df.categories[i])
    inter = list(set(cats) & set(cat_list))
    if inter != []:
        for j in inter:
            cat_df[j][i] = 1

0


/Users/ethanweber/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000


In [50]:
from scipy.spatial.distance import cosine

i = 5
j = 720
cat_df.iloc[1,0]
({'biz_1': cat_df.iloc[i,0], 'biz_2': cat_df.iloc[j,0], 'Correlation': (cosine(cat_df.iloc[i,1:], cat_df.iloc[j,1:]))})

{'Correlation': 1.0,
 'biz_1': '3LWsVfsSmb_Nzbi2YQ-NIA',
 'biz_2': '-wDNRL4lm1UgeFLG9KcrjQ'}

In [53]:
from math import*

def euclidean_distance(x,y):

    return sqrt(sum(pow(a-b,2) for a, b in zip(x, y)))

print euclidean_distance(cat_df.iloc[i,1:], cat_df.iloc[j,1:])

3.0


In [48]:
cat_df.iloc[i,1:], cat_df.iloc[j,1:]

(Buffets                      1
 Creperies                    0
 Juice Bars & Smoothies       0
 Grocery                      0
 Delis                        0
 Halal                        1
 Cantonese                    0
 Noodles                      0
 Chicken Wings                0
 Food Stands                  0
 Lounges                      0
 Cajun/Creole                 0
 Greek                        0
 Chinese                      0
 Food Trucks                  0
 Middle Eastern               0
 Diners                       0
 Sports Bars                  0
 Vegetarian                   1
 Chicken Shop                 0
 Taiwanese                    0
 Vietnamese                   0
 Pizza                        0
 Poke                         0
 Food Court                   0
 Music Venues                 0
 Seafood                      0
 Japanese                     0
 Hot Dogs                     0
 Salvadoran                   0
                             ..
 Casinos

In [35]:
del cat_df['categories']
del cat_df['big_cat']

In [37]:
cat_df.to_pickle('categories_df.pkl')

In [67]:
import csv
with open('/Users/ethanweber/Desktop/categ_corr.csv', 'w') as csvfile:
    fieldnames = ['biz_1','biz_2', 'Euclidean']
    writer = csv.DictWriter(csvfile, delimiter="|", fieldnames=fieldnames)
    writer.writeheader()
    top_biz = []
    for i in range(4092):
        for j in range((i+1), 4093):
            writer.writerow({'biz_1': cat_df.iloc[i,0], 'biz_2': cat_df.iloc[j,0], 'Euclidean': euclidean_distance(cat_df.iloc[i,1:], cat_df.iloc[j,1:])})


In [56]:
def jaccard_similarity(x,y):

    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    return intersection_cardinality/float(union_cardinality)


In [65]:
for i in range(20):
    for j in range((i+1), 20):
        #print({'biz_1': cat_df.iloc[i,0], 'biz_2': cat_df.iloc[j,0], 'euclidean': cosine(cat_df.iloc[i,1:], cat_df.iloc[j,1:])})
        print ({'biz_1': cat_df.iloc[i,0], 'biz_2': cat_df.iloc[j,0], 'Euclidean': euclidean_distance(cat_df.iloc[i,1:], cat_df.iloc[j,1:])})
        #print ({cat_df.iloc[i,0], cat_df.iloc[j,0], 'jaccard': jaccard_similarity(cat_df.iloc[i,1:], cat_df.iloc[j,1:])})

{'biz_2': 'P7pxQFqr7yBKMMI2J51udw', 'biz_1': 'CiYLq33nAyghFkUR15pP-Q', 'Correlation': 2.449489742783178}
{'biz_2': 'g8OnV26ywJlZpezdBnOWUQ', 'biz_1': 'CiYLq33nAyghFkUR15pP-Q', 'Correlation': 1.7320508075688772}
{'biz_2': 'gTlDDzDEHyDQ6iwjNhpI6A', 'biz_1': 'CiYLq33nAyghFkUR15pP-Q', 'Correlation': 1.4142135623730951}
{'biz_2': '-WLrZPzjKfrftLWaCi1QZQ', 'biz_1': 'CiYLq33nAyghFkUR15pP-Q', 'Correlation': 1.7320508075688772}
{'biz_2': '3LWsVfsSmb_Nzbi2YQ-NIA', 'biz_1': 'CiYLq33nAyghFkUR15pP-Q', 'Correlation': 2.6457513110645907}
{'biz_2': 'AnXovUgUHJap3QaDRnjL1w', 'biz_1': 'CiYLq33nAyghFkUR15pP-Q', 'Correlation': 2.0}
{'biz_2': 'd_L-rfS1vT3JMzgCUGtiow', 'biz_1': 'CiYLq33nAyghFkUR15pP-Q', 'Correlation': 1.4142135623730951}
{'biz_2': 'LYWbyu1EQM2o5hwHG5_Ldg', 'biz_1': 'CiYLq33nAyghFkUR15pP-Q', 'Correlation': 1.7320508075688772}
{'biz_2': 'ujHiaprwCQ5ewziu0Vi9rw', 'biz_1': 'CiYLq33nAyghFkUR15pP-Q', 'Correlation': 1.7320508075688772}
{'biz_2': 'cYwJA2A6I12KNkm2rtXd5g', 'biz_1': 'CiYLq33nAyghFkUR

In [ ]:
flatten = lambda l: [item for sublist in l for item in sublist]
joiner = lambda x: ' '.join(flatten(x))

In [ ]:
rest_2['text2'] = map(joiner, rest_2['text2'])

In [ ]:
tfidf_df = rest_2

In [ ]:
tfidf_df['biz_ID'] = tfidf_df.index

In [ ]:
tfidf_df.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups

In [ ]:
n_features = 1000

tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=n_features,
                                stop_words='english')
tf = tf_vectorizer.fit_transform(tfidf_df['text2'])
TF = TfidfTransformer()
tf_idf = TF.fit_transform(tf)
tf_idf = tf_idf.toarray() #convert sparse matrix to array
#tf_idf.toarray()[:100, :100]

In [ ]:
index_list = list(final_df.index)
list_ij = []

cosine_df = pd.DataFrame(columns=['biz_1', 'biz_2'])
for i in range(4781):
    print i
    for j in range((i+1), 4782):
        list_ij.append({'biz_1': index_list[i], 'biz_2': index_list[j]})
        #cosine_df = cosine_df.append({'biz_1': index_list[i], 'biz_2': index_list[j]}, ignore_index = True)


In [ ]:
import csv
with open("/Users/apple/Desktop/capstone/model/pred_restaurant_pearsonr.csv", 'w') as csvfile:
    fieldnames = ['biz1','biz2', 'Correlation']
    writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
    writer.writeheader()
    top_biz = []
    for i in most_reviewed_biz.business_id:
        top_biz.append(i)
    for biz1 in top_biz:
    #print "Calculating the correlations for:", biz1
        for biz2 in top_biz:
            if biz1 != biz2:
                writer.writerow({'biz1':biz1,'biz2':biz2,'Correlation':calculate_correlation(biz1, biz2)})    

In [ ]:
i= 5
j= 6
type(final_df.iloc[i ,1000])

In [ ]:
str(final_df.iloc[i ,1000]), str(final_df.iloc[j ,1000]), cosine(final_df.iloc[i,1:1000], final_df.iloc[j,1:1000])

In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cosine

#cosine(final_df.iloc[0], final_df.iloc[1])
#[final_df.iloc[i, 1000], final_df.iloc[j, 1000]]

import csv
with open('/Users/ethanweber/Desktop/cosine_corr.csv', 'w') as csvfile:
    fieldnames = ['biz_1','biz_2', 'Correlation']
    writer = csv.DictWriter(csvfile, delimiter="|", fieldnames=fieldnames)
    writer.writeheader()
    top_biz = []
    for i in range(4781):
        for j in range((i+1), 4782):
            writer.writerow({'biz_1': final_df.iloc[i ,1000], 'biz_2': final_df.iloc[j ,1000], 'Correlation': cosine(final_df.iloc[i , 1:1000], final_df.iloc[j, 1:1000])})

In [ ]:
#final_df = pd.DataFrame(tf_idf, columns = tf_vectorizer.get_feature_names())
final_df['names'] = pd.Series(list(tfidf_df['biz_ID']))
#trimDF_train = pd.concat([tf_idf.drop(['text2'], axis=1), tf_idf.add_prefix('tf_idf')], axis=1)

In [ ]:
final_df = final_df.iloc[:, range(1000)]

In [ ]:
#lv_df['categories'] = map(fix_categories, lv_df['categories'])


In [ ]:
#catlist = ['Active Life', 'Arts & Entertainment', 'Automotive', 'Beauty & Spas', 'Education', 'Event Planning & Services', 'Financial Services', 'Food', 'Home Services', 'Health & Medical', 'Hotels & Travel', 'Local Flavor', 'Local Services', 'Mass Media', 'Nightlife', 'Pets', 'Professional Services', 'Public Services & Government', 'Real Estate', 'Religious Organizations', 'Restaurants', 'Shopping']
#rmlist = ['Automotive', 'Education', 'Financial Services',  'Health & Medical', 'Mass Media', 'Home Services', 'Local Services', 'Pets', 'Professional Services', 'Public Services & Government', 'Real Estate', 'Religious Organizations']
#new_list = ['Active Life', 'Arts & Entertainment', 'Beauty & Spas', 'Event Planning & Services', 'Food', 'Hotels & Travel', 'Local Flavor', 'Nightlife', 'Restaurants', 'Shopping']
#print len(sorted((rmlist) + (new_list)))
#print len(sorted(catlist))

In [ ]:
#lv_df.to_pickle('lv_df.pkl')  # where to save it, usually as a .pkl

#lv_df = pd.read_pickle('lv_df.pkl')


In [ ]:
type(lv_df.big_cat[3])

In [ ]:
restaurant_df

In [ ]:
max(final_df.iloc[1])

In [ ]:
active_df = lv_df2[map(lambda x: 'Active Life' in x, lv_df2['big_cat'])]
arts_df = lv_df2[map(lambda x: 'Arts & Entertainment' in x, lv_df2['big_cat'])]
beauty_df = lv_df2[map(lambda x: 'Beauty & Spas' in x, lv_df2['big_cat'])]
events_df = lv_df2[map(lambda x: 'Event Planning & Services' in x, lv_df2['big_cat'])]
food_df = lv_df2[map(lambda x: 'Food' in x, lv_df2['big_cat'])]
hotels_df = lv_df2[map(lambda x: 'Hotels & Travel' in x, lv_df2['big_cat'])]
local_df = lv_df2[map(lambda x: 'Local Flavor' in x, lv_df2['big_cat'])]
nightlife_df = lv_df2[map(lambda x: 'Nightlife' in x, lv_df2['big_cat'])]
restaurant_df = lv_df2[map(lambda x: 'Restaurants' in x, lv_df2['big_cat'])]
shops_df = lv_df2[map(lambda x: 'Shopping' in x, lv_df2['big_cat'])]

In [ ]:
active_df.to_csv('active_df.csv', sep='|', encoding='utf-8')
arts_df.to_csv('arts_df.csv', sep='|', encoding='utf-8')
beauty_df.to_csv('beauty_df.csv', sep='|', encoding='utf-8')
events_df.to_csv('events_df.csv', sep='|', encoding='utf-8')
food_df.to_csv('food_df.csv', sep='|', encoding='utf-8')
hotels_df.to_csv('hotels_df.csv', sep='|', encoding='utf-8')
local_df.to_csv('local_df.csv', sep='|', encoding='utf-8')
nightlife_df.to_csv('nightlife_df.csv', sep='|', encoding='utf-8')
restaurant_df.to_csv('restaurant_df.csv', sep='|', encoding='utf-8')
shops_df.to_csv('shops_df.csv', sep='|', encoding='utf-8')

In [ ]:
active_df.to_pickle('./pickles/active_df.pkl')
arts_df.to_pickle('./pickles/arts_df.pkl')
beauty_df.to_pickle('./pickles/beauty_df.pkl')
events_df.to_pickle('./pickles/events_df.pkl')
food_df.to_pickle('./pickles/food_df.pkl')
hotels_df.to_pickle('./pickles/hotels_df.pkl')
local_df.to_pickle('./pickles/local_df.pkl')
nightlife_df.to_pickle('./pickles/nightlife_df.pkl')
restaurant_df.to_pickle('./pickles/restaurant_df.pkl')
shops_df.to_pickle('./pickles/shops_df.pkl')

In [ ]:
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO  # ipython sometimes messes up the logging setup; restore

In [ ]:
restaurant_df = lv_df[map(lambda x: 'Restaurants' in x, lv_df2['big_cat'])]

In [ ]:
restaurant_df

In [ ]:
n_features = 1000
n_topics = 10
n_top_words = 20

In [ ]:
tf = tf_vectorizer.fit_transform(text_train)
TF = TfidfTransformer()
tf_idf = TF.fit_transform(tf)
tf_idf = tf_idf.toarray() #convert sparse matrix to array
#tf_idf.toarray()[:100, :100]

In [ ]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=n_features,
                                stop_words='english')

In [ ]:
def iter_wiki(dump_file):
    """Yield each article from the Wikipedia dump, as a `(title, tokens)` 2-tuple."""
    ignore_namespaces = 'Wikipedia Category File Portal Template MediaWiki User Help Book Draft'.split()
    for title, text, pageid in _extract_pages(smart_open(dump_file)):
        text = filter_wiki(text)
        tokens = tokenize(text)
        if len(tokens) < 50 or any(title.startswith(ns + ':') for ns in ignore_namespaces):
            continue  # ignore short articles and various meta-articles
        yield title, tokens

In [ ]:
tf_idf

In [ ]:

tf = tf_vectorizer.fit_transform(text_train)
TF = TfidfTransformer()
tf_idf = TF.fit_transform(tf)
tf_idf = tf_idf.toarray() #convert sparse matrix to array
#tf_idf.toarray()[:100, :100]

In [ ]:
tf_idf

In [ ]:
#LV_samplei.categories = map(lambda x: fix_categories(str(x)) , LV_samplei.categories)

In [ ]:
import gensim
from gensim import corpora
df_list = [active_df, arts_df, beauty_df, events_df, food_df, hotels_df, local_df, nightlife_df, restaurant_df, shops_df]

In [ ]:
Lda = gensim.models.ldamodel.LdaModel

In [ ]:
active_dict = corpora.Dictionary(active_df.categories)
arts_dict = corpora.Dictionary(arts_df.categories)
beauty_dict = corpora.Dictionary(beauty_df.categories)
events_dict = corpora.Dictionary(events_df.categories)
food_dict = corpora.Dictionary(food_df.categories)
hotels_dict = corpora.Dictionary(hotels_df.categories)
local_dict = corpora.Dictionary(local_df.categories)
nightlife_dict = corpora.Dictionary(nightlife_df.categories)
restaurant_dict = corpora.Dictionary(restaurant_df.categories)
shops_dict = corpora.Dictionary(shops_df.categories)

In [ ]:
doc_term_active = [active_dict.doc2bow(x) for x in active_df.categories]
doc_term_arts = [arts_dict.doc2bow(x) for x in arts_df.categories]
doc_term_beauty = [beauty_dict.doc2bow(x) for x in beauty_df.categories]
doc_term_events = [events_dict.doc2bow(x) for x in events_df.categories]
doc_term_food = [food_dict.doc2bow(x) for x in food_df.categories]
doc_term_hotels = [hotels_dict.doc2bow(x) for x in hotels_df.categories]
doc_term_local = [local_dict.doc2bow(x) for x in local_df.categories]
doc_term_nightlife = [nightlife_dict.doc2bow(x) for x in nightlife_df.categories]
doc_term_restaurant = [restaurant_dict.doc2bow(x) for x in restaurant_df.categories]
doc_term_shops = [shops_dict.doc2bow(x) for x in shops_df.categories]

In [ ]:
lda_active = Lda(doc_term_active, num_topics=3, id2word = active_dict, passes=3)
print 'active done'
lda_arts = Lda(doc_term_arts, num_topics=5, id2word = arts_dict, passes=3)
print 'arts done'
lda_beauty = Lda(doc_term_beauty, num_topics=3, id2word = beauty_dict, passes=3)
print 'beauty done'
lda_events = Lda(doc_term_events, num_topics=7, id2word = events_dict, passes=3)
print 'events done'
lda_food = Lda(doc_term_food, num_topics=10, id2word = food_dict, passes=3)
print 'food done'
lda_hotels = Lda(doc_term_hotels, num_topics=6, id2word = hotels_dict, passes=3)
print 'hotels done'
lda_local = Lda(doc_term_local, num_topics=3, id2word = local_dict, passes=3)
print 'local done'
lda_nightlife = Lda(doc_term_nightlife, num_topics=3, id2word = nightlife_dict, passes=3)
print 'nightlife done'
lda_restaurant = Lda(doc_term_restaurant, num_topics=45, id2word = restaurant_dict, passes=3)
print 'restaurants done'
lda_shops = Lda(doc_term_shops, num_topics=20, id2word = shops_dict, passes=3)
print 'shops done'

In [ ]:
lda_active.save('./lda_pickles/lda_active.pkl')
lda_arts.save('./lda_pickles/lda_arts.pkl')
lda_beauty.save('./lda_pickles/lda_beauty.pkl')
lda_events.save('./lda_pickles/lda_events.pkl')
lda_food.save('./lda_pickles/lda_food.pkl')
lda_hotels.save('./lda_pickles/lda_hotels.pkl')
lda_local.save('./lda_pickles/lda_local.pkl')
lda_nightlife.save('./lda_pickles/lda_nightlife.pkl')
lda_restaurant.save('./lda_pickles/lda_restaurant.pkl')
lda_shops.save('./lda_pickles/lda_shops.pkl')

In [ ]:
lda_list = [lda_active, lda_arts, lda_beauty, lda_events, lda_food, lda_hotels, lda_local, lda_nightlife, lda_restaurant, lda_shops] 

In [ ]:
for i in lda_list:
    print i
    print i.print_topics(num_topics=3, num_words=3)[0]

In [ ]:
for i in range(10):
    for j in range(i+1, (10)):
        print i, j
    

In [ ]:
for i in range(len())

In [ ]:
# As I want to avoid dealing with a huge sparse matrix, I will only select the top most reviewed books for our example 

most_reviewed_biz = pd.DataFrame({'count' : LV_test.groupby(["name_x"]).size()})\
                                    .reset_index().sort(['count'],ascending = False)

most_reviewed_biz.head(20) 

In [ ]:
# Let's create a function that collect the reviews of our common reviewers
def get_biz_reviews(biz, common_reviewers):
    mask = (LV_test.user_id.isin(common_reviewers)) & (LV_test.name_x==biz)
    reviews = LV_test[mask].sort('user_id')
    reviews = reviews[reviews.user_id.duplicated()==False]
    return reviews

In [ ]:
# Importing the scipy library to measure the pearson correlation coefficient
from scipy.stats.stats import pearsonr
def calculate_correlation(biz1, biz2):
    # We start by finding the common reviewers
    biz_1_reviewers = LV_test[LV_test.name_x == biz1].user_id
    biz_2_reviewers = LV_test[LV_test.name_x == biz2].user_id
    common_reviewers = set(biz_1_reviewers).intersection(biz_2_reviewers)

    # Then we look for the reviews given by common reviewers
    biz_1_reviews = get_biz_reviews(biz1, common_reviewers)
    biz_2_reviews = get_biz_reviews(biz2, common_reviewers)
    
    # Calculate the Pearson Correlation Score
    return pearsonr(biz_1_reviews.stars_x, biz_2_reviews.stars_x)[0]

In [ ]:
# Getting the list of the most reviewed biz

top_biz = []

for i in most_reviewed_biz.name_x[0:13]:
    top_biz.append(i)

    
# calculate the correlation for our top biz
correlation_coefficient = []

for biz1 in top_biz:
    print "Calculating the correlations for:", biz1
    for biz2 in top_biz:
        if biz1 != biz2:
            row = [biz1, biz2] + [calculate_correlation(biz1, biz2)]
            correlation_coefficient.append(row)

In [ ]:
# Let's look at what the table of correlation looks like
cols = ["biz_1", "biz_2", "Correlation"]
correlation_coefficient = pd.DataFrame(correlation_coefficient, columns=cols).sort('Correlation')
correlation_coefficient.head(10)

In [ ]:
def calc_correlation(corr, biz1, biz2):
    mask = (corr.biz_1==biz1) & (corr.biz_2==biz2)
    row = corr[mask]
    corr = row
    return corr.sum(axis=1).tolist()[0]

In [ ]:
calc_correlation(correlation_coefficient,"McDonald's", "Smashburger")